# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 9

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 593145bf-dcd9-4fbb-8e4f-f065c7fed3ca successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (208, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID a471528b-2d8c-4608-b475-671f308b311f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (207, 3)


,mm,LoanProduct,SubmittedApps
61,2023-12-01,Flex-up,421
63,2023-12-01,Reloan,368
92,2023-07-01,Quick,27147
206,NaT,SIL-Online(discontinued),7421
123,2023-02-01,Reloan,90
66,2023-11-01,Flex,1874
144,2022-10-01,SIL-Instore,187
161,2022-06-01,SIL-Instore,124
50,2024-02-01,Quick,18650
76,2023-10-01,SIL-Instore,6461


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID ad6877c2-6c47-4c47-8fc9-a988dbf08c0a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (169, 3)


,mm,LoanProduct,ApprovedApps
0,2024-10-01,Flex-up,31
1,2024-10-01,Quick,222
2,2024-10-01,Reloan,44
3,2024-10-01,SIL ZERO,86
4,2024-10-01,SIL-Instore,849
5,2024-09-01,Flex-up,514
6,2024-09-01,Quick,3370
7,2024-09-01,Reloan,533
8,2024-09-01,SIL ZERO,1724
9,2024-09-01,SIL-Instore,13340


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID 4bb70688-73c6-419b-8f35-dc40a0b633bc successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (168, 3)


,mm,LoanProduct,BookedApps
0,2024-10-01,Flex-up,60
1,2024-10-01,Quick,226
2,2024-10-01,Reloan,44
3,2024-10-01,SIL ZERO,84
4,2024-10-01,SIL-Instore,833
5,2024-09-01,Flex-up,651
6,2024-09-01,Quick,3302
7,2024-09-01,Reloan,533
8,2024-09-01,SIL ZERO,1717
9,2024-09-01,SIL-Instore,13295


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID d94a90c4-e298-4a4c-9104-ce25db44f611 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (168, 3)


,mm,LoanProduct,BookedAmt
0,2024-10-01,Flex-up,2.055471e+06
1,2024-10-01,Quick,2.862610e+06
2,2024-10-01,Reloan,9.070220e+05
3,2024-10-01,SIL ZERO,1.108485e+06
4,2024-10-01,SIL-Instore,8.273011e+06
5,2024-09-01,Flex-up,2.255746e+07
6,2024-09-01,Quick,4.187505e+07
7,2024-09-01,Reloan,1.239983e+07
8,2024-09-01,SIL ZERO,2.295222e+07
9,2024-09-01,SIL-Instore,1.243032e+08


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           168 non-null    datetime64[us]
 1   LoanProduct  168 non-null    object        
 2   BookedAmt    168 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 4.1+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
200,2024-09-01,Flex-up,514,514,514,651,2.255746e+07
201,2024-09-01,Quick,55152,45317,3370,3302,4.187505e+07
202,2024-09-01,Reloan,591,563,533,533,1.239983e+07
203,2024-09-01,SIL ZERO,3391,3346,1724,1717,2.295222e+07
204,2024-09-01,SIL-Instore,22204,21938,13340,13295,1.243032e+08
205,2024-10-01,Flex-up,31,31,31,60,2.055471e+06
206,2024-10-01,Quick,4458,3624,222,226,2.862610e+06
207,2024-10-01,Reloan,57,55,44,44,9.070220e+05
208,2024-10-01,SIL ZERO,175,175,86,84,1.108485e+06
209,2024-10-01,SIL-Instore,1501,1482,849,833,8.273011e+06


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2024-09-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
54,2022-08-01,Flex,5558,4732,594,599,1.198500e+07
55,2022-08-01,Quick,43764,30660,1910,1926,3.300100e+07
56,2022-08-01,SIL-Instore,967,956,456,457,1.137941e+07
57,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1.197499e+06
58,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
200,2024-09-01,Flex-up,514,514,514,651,2.255746e+07
201,2024-09-01,Quick,55152,45317,3370,3302,4.187505e+07
202,2024-09-01,Reloan,591,563,533,533,1.239983e+07
203,2024-09-01,SIL ZERO,3391,3346,1724,1717,2.295222e+07


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 6b25e076-370f-4ea5-a951-36e00b3a7a43 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (207427, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
207417,60828972170012,SIL-Instore,2024-09-30 15:29:30,None
207418,60828980740019,SIL-Instore,2024-09-30 18:27:04,None
207419,60828982490014,SIL-Instore,2024-09-30 19:14:32,None
207420,60828994950011,SIL-Instore,2024-10-01 10:49:47,None
207421,60829003890014,SIL-Instore,2024-10-01 14:44:40,None
207422,60829005570012,SIL-Instore,2024-10-01 15:15:06,None
207423,60829006580016,SIL-Instore,2024-10-01 15:42:57,None
207424,60829011100019,SIL-Instore,2024-10-01 17:52:29,None
207425,60829042300018,SIL-Instore,2024-10-02 17:35:48,None
207426,60829043880019,SIL-Instore,2024-10-02 18:13:53,None


In [15]:
loan_type[loan_type['loanAccountNumber']=='60828136490016']

,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
165997,60828136490016,SIL-Instore,2024-09-01 12:25:12,None


In [16]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [17]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [18]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID 51132f43-a3cb-42bd-ad53-835d6195ad69 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (168, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2024,10,Flex-up,60,24
1,2024,10,Quick,226,131
2,2024,10,Reloan,44,26
3,2024,10,SIL ZERO,84,62
4,2024,10,SIL-Instore,833,568
5,2024,9,Flex-up,651,215
6,2024,9,Quick,3302,1987
7,2024,9,Reloan,533,149
8,2024,9,SIL ZERO,1717,1176
9,2024,9,SIL-Instore,13295,9134


In [19]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [20]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)